# <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2; text-align: center;">Topic Modeling</div>

An important task coming after data preprocessing and analysis (being the second step to virtually any NLP project) is topic modeling: a process that separates the existing data into multiple clusters, each of them representing a different topic. This is a crucial step in the process of understanding the data and extracting valuable insights from it. For this task, the team decided to use the BERTopic library, which is a topic modeling technique that leverages transformers model to create dense representations of the documents and then clusters them using HDBSCAN.

#### Used Embeddings

The embeddings used for topic modeling are taken from the project of digitalepidemiologylab in GitHub, which generated embeddings from COVID-19 tweets. These Embeddings are related to the BERT model, and a description about them can be found in the [official repository of digitalepidemoloylab's project](https://github.com/digitalepidemiologylab/covid-twitter-bert).

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Dependencies Imports</div>

In [ ]:
!pip install umap-learn
!pip install hdbscan
!pip install sentence-transformers
!pip install bertopic
!pip install nltk
!pip install nbformat
!pip install datamapplot
!pip install dask[dataframe]

In [17]:
import pandas as pd
import html
from umap import UMAP
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import datamapplot
import re

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Preprocessing</div>

Before starting to transform the data, some important preprocessing steps must be done in order to clean the data and maintain coherency in results with what has been done in the previous notebooks. The following steps were taken:

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Data Loading</div>

In [18]:
DATA = 'data/'
test = pd.read_csv(DATA + 'Constraint_English_Test.csv', delimiter=';', encoding='utf-8')
train = pd.read_csv(DATA + 'Constraint_English_Train.csv', delimiter=';', encoding='utf-8')
val = pd.read_csv(DATA + 'Constraint_English_Val.csv', delimiter=';', encoding='utf-8')

tweets = pd.concat([train, val, test], ignore_index=True)
tweets.drop(columns=['id'], inplace=True)
tweets['tweet'] = tweets['tweet'].apply(html.unescape)


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Data Cleaning</div>

In [19]:
processed_tweets = tweets.copy()
processed_tweets = processed_tweets.drop_duplicates(subset='tweet', keep='first')

# remove links
def filter(tweet:str):
    # https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url#3809435
    return re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)','',tweet)

processed_tweets['tweet_nolinks'] = processed_tweets['tweet'].apply(filter)

# Recalculate indexes
processed_tweets['tweet'].reset_index(drop=True, inplace=True)

processed_tweets

,tweet,label,tweet_nolinks
0,The CDC currently reports 99031 deaths. In gen...,real,The CDC currently reports 99031 deaths. In gen...
1,States reported 1121 deaths a small rise from ...,real,States reported 1121 deaths a small rise from ...
2,Politically Correct Woman (Almost) Uses Pandem...,fake,Politically Correct Woman (Almost) Uses Pandem...
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real,#IndiaFightsCorona: We have 1524 #COVID testin...
4,Populous states can generate large case counts...,real,Populous states can generate large case counts...
...,...,...,...
10695,#CoronaVirusUpdates: State-wise details of Tot...,real,#CoronaVirusUpdates: State-wise details of Tot...
10696,Tonight 12(midnight) onwards Disaster Manageme...,fake,Tonight 12(midnight) onwards Disaster Manageme...
10697,296 new cases of #COVID19Nigeria; Plateau-85 E...,real,296 new cases of #COVID19Nigeria; Plateau-85 E...
10698,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...,real,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...


## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Data Transformation (Embeddings)</div>

In order to extract the different topics, a set of pre-trained models, called embeddings, can be used to transform the data into a format that can be used by the topic modeling algorithm. The adecuacy of the embeddings to the domain of the data is paramount for the well-performing of the topic modeling algorithm. In this case, the embeddigs used will be `covid-twitter-bert`, a model provided by digitalepidemiologylab.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Load Embeddings</div>

In [20]:
from sentence_transformers import SentenceTransformer
# Load the pre-trained model
sentence_model = SentenceTransformer('digitalepidemiologylab/covid-twitter-bert-v2')

# Encode the tweets
#embeddings = sentence_model.encode(processed_tweets["tweet"], show_progress_bar=True)

No sentence-transformers model found with name digitalepidemiologylab/covid-twitter-bert-v2. Creating a new one with mean pooling.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=None, low_memory=False)

In [22]:
# Step 3 - Cluster reduced embeddings
cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [23]:
# Step 4 - bag-of-words representation
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1,2))

In [24]:
# Step 3 - Cluster reduced embeddings
cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [25]:
# Step 5 - Create topic representation (extract topic words)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [26]:
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model
representation_model = KeyBERTInspired()

In [47]:
# All steps together
topic_model_real = BERTopic(
  embedding_model=sentence_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english'
)

umap_model_fake = UMAP(n_neighbors=min(len([tweet for tweet in processed_tweets[processed_tweets["label"] == "fake"]["tweet"]]) - 1, 15), n_components=10, metric='cosine', random_state=None, low_memory=False)
topic_model_fake = BERTopic(
  embedding_model=sentence_model,          # Step 1 - Extract embeddings
  umap_model=umap_model_fake,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english'
)

In [48]:
# Trainning process
topics_real, probs_real = topic_model_real.fit_transform([tweet for tweet in processed_tweets[processed_tweets['label'] == 'real']['tweet']])

2025-01-07 00:38:29,256 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 175/175 [14:18<00:00,  4.91s/it]
2025-01-07 00:52:47,851 - BERTopic - Embedding - Completed ✓
2025-01-07 00:52:47,852 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-07 00:52:54,383 - BERTopic - Dimensionality - Completed ✓
2025-01-07 00:52:54,384 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-07 00:52:55,042 - BERTopic - Cluster - Completed ✓
2025-01-07 00:52:55,045 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-07 00:55:41,672 - BERTopic - Representation - Completed ✓


In [49]:
topics_fake, probs_fake = topic_model_fake.fit_transform([tweet for tweet in processed_tweets[processed_tweets["label"] == "fake"]["tweet"]])

2025-01-07 00:55:41,900 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 160/160 [08:18<00:00,  3.12s/it]
2025-01-07 01:04:00,766 - BERTopic - Embedding - Completed ✓
2025-01-07 01:04:00,767 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-07 01:04:06,849 - BERTopic - Dimensionality - Completed ✓
2025-01-07 01:04:06,850 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-07 01:04:07,378 - BERTopic - Cluster - Completed ✓
2025-01-07 01:04:07,381 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-07 01:06:53,941 - BERTopic - Representation - Completed ✓


In [50]:
# Topics visualization
topic_model_real.visualize_topics()

In [51]:
# hierarchical topic visualization
topic_model_real.visualize_hierarchy()

In [52]:
topic_model_fake.visualize_topics()

In [53]:
topics_fake

[14,
 -1,
 0,
 -1,
 2,
 0,
 -1,
 -1,
 -1,
 -1,
 19,
 -1,
 -1,
 5,
 20,
 -1,
 16,
 -1,
 -1,
 4,
 20,
 13,
 7,
 -1,
 -1,
 8,
 5,
 33,
 15,
 1,
 6,
 12,
 31,
 -1,
 27,
 -1,
 -1,
 5,
 9,
 -1,
 -1,
 23,
 1,
 -1,
 7,
 -1,
 12,
 16,
 26,
 6,
 5,
 43,
 15,
 3,
 11,
 -1,
 5,
 20,
 -1,
 -1,
 8,
 -1,
 2,
 3,
 30,
 7,
 -1,
 -1,
 -1,
 17,
 0,
 20,
 -1,
 12,
 17,
 1,
 -1,
 3,
 16,
 0,
 -1,
 19,
 5,
 3,
 20,
 -1,
 -1,
 10,
 -1,
 0,
 -1,
 5,
 0,
 9,
 34,
 37,
 -1,
 7,
 7,
 -1,
 -1,
 -1,
 26,
 -1,
 2,
 5,
 -1,
 -1,
 1,
 34,
 6,
 -1,
 23,
 -1,
 -1,
 -1,
 0,
 -1,
 8,
 -1,
 -1,
 -1,
 19,
 10,
 34,
 1,
 12,
 11,
 15,
 33,
 52,
 5,
 46,
 0,
 1,
 -1,
 1,
 35,
 5,
 6,
 11,
 24,
 -1,
 24,
 -1,
 -1,
 -1,
 4,
 9,
 14,
 0,
 -1,
 -1,
 -1,
 -1,
 8,
 1,
 -1,
 2,
 9,
 -1,
 -1,
 -1,
 10,
 13,
 35,
 -1,
 -1,
 -1,
 -1,
 4,
 1,
 14,
 3,
 13,
 1,
 1,
 0,
 -1,
 50,
 2,
 -1,
 22,
 11,
 5,
 49,
 -1,
 -1,
 8,
 28,
 -1,
 5,
 -1,
 -1,
 1,
 -1,
 -1,
 7,
 27,
 12,
 -1,
 1,
 1,
 17,
 -1,
 -1,
 5,
 -1,
 21,
 17,
 -1,
 2,
 -1,
 8,
 

In [54]:
topic_model_fake.visualize_hierarchy()